In [1]:
import pandas as pd

path = "data/mapdataall.csv"

fire_data = pd.read_csv(path)

# Columns to include:
# incident_name  
# incident_id
# incident_county 
# incident_acres_burned
# incident_dateonly_extinguished
# incident_dateonly_created
# incident_longitude
# incident_latitude

fire_data_columns = ["incident_name", "incident_id", "incident_county", 
                     "incident_longitude", "incident_latitude", "incident_dateonly_created", 
                     "incident_dateonly_extinguished", "incident_acres_burned"]

fire_data_filtered = fire_data[fire_data_columns]
fire_data_filtered["incident_dateonly_created"] = pd.to_datetime(fire_data_filtered["incident_dateonly_created"])
fire_data_filtered["incident_dateonly_extinguished"] = pd.to_datetime(fire_data_filtered["incident_dateonly_extinguished"])
fire_data_filtered["year_extinguished"] = fire_data_filtered["incident_dateonly_extinguished"].dt.year
fire_data_filtered = fire_data_filtered.sort_values("year_extinguished", ascending = True, ignore_index = True)
fire_data_filtered = fire_data_filtered.loc[fire_data_filtered["year_extinguished"] > 2010]

fire_data_filtered


/Users/jakepohs/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jakepohs/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jakepohs/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,incident_name,incident_id,incident_county,incident_longitude,incident_latitude,incident_dateonly_created,incident_dateonly_extinguished,incident_acres_burned,year_extinguished
3,Dersch Fire,29a2efda-c28f-4a41-9c6d-8cb8f763d464,Shasta,-122.140914,40.483662,2013-11-07,2013-11-07,33.0,2013.0
4,Sharp Fire,62eabea3-40be-497d-94e4-150f011b925f,San Bernardino,-117.620000,34.350000,2013-08-08,2013-08-13,243.0,2013.0
5,Cloverdale Fire,0537a175-8fe9-4f5f-b56f-77848fa195f4,Shasta,-122.482590,40.473356,2013-08-14,2013-08-14,27.0,2013.0
6,Swedes Fire,0758e1c0-95ea-47e7-b5c1-5f3ea96bc664,Butte,-121.382360,39.446268,2013-08-16,2013-08-22,2264.0,2013.0
7,American Fire,8213f5c7-34fa-403b-a4bc-da2ace6e6625,Placer,-120.650000,39.120000,2013-08-10,2013-08-30,27440.0,2013.0
...,...,...,...,...,...,...,...,...,...
1864,Graham Fire,58ba3463-14d8-4ac7-86c3-aaeb920de442,Tehama,-121.776000,40.050000,2022-06-10,2022-06-16,100.0,2022.0
1865,Park Fire,9e92e949-1e1c-47b7-9a46-71664466a15c,Butte,-121.431372,39.385829,2022-06-09,2022-06-11,54.0,2022.0
1866,Hesperia Fire,968e2f0c-5255-4a21-b166-c9a06a645825,San Bernardino,-117.346350,34.312670,2022-06-06,2022-06-10,95.0,2022.0
1867,Edgewood,eb1d79f9-a00a-4fc8-b979-5fb00e24da0b,Santa Cruz,-122.277420,37.459222,2022-06-21,2022-06-26,20.0,2022.0
